# 모듈 임포트

In [10]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
import random
import torch.optim as optim
from torch.optim import AdamW
from torch import nn
from torch.utils.data import DataLoader, Dataset,TensorDataset
from torch.nn import functional as F
import re
import urllib.request
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from konlpy.tag import Mecab
import wandb 
import collections
import pickle
import kss
from hanspell import spell_checker

from ktextaug import TextAugmentation
import ktextaug

import os
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForMaskedLM
from transformers import GPT2Config, GPT2Tokenizer, GPT2LMHeadModel

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'NanumGothic'

random_seed = 42 
random.seed(random_seed)
np.random.seed(random_seed)
os.environ["PYTHONHASHSEED"] = str(random_seed)
tf.random.set_seed(random_seed)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(random_seed)
if device == 'cuda':
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
wandb.login()

# 1. 데이터 불러오기

In [92]:
def load_data(file_name):#파일이름만 넣기
    data_path = os.getenv('HOME') + '/aiffel/project/FnGuide/data/ad_news'#경로는 자신의 경로로 바꿔서 하면 됨
    file_path = os.path.join(data_path, file_name)
    df = pd.read_csv(file_path, encoding='utf-8', delimiter='\t')
    return df

## 1-1 사전학습 모델 및 토크나이저 불러오기

In [35]:
model_name = "klue/roberta-base"  # 허깅페이스에서 모델만 갈아끼우면 됨

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 예측하고자 하는 클래스의 수를 넣어줘야함
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'class

In [5]:
# kb-albert-char-base-v2, kfdeberta-base
model_name='kfdeberta-base'

p_model_path = os.getenv('HOME') + "/aiffel/project/FnGuide/kfdeberta-base"

model = AutoModelForSequenceClassification.from_pretrained(p_model_path, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(p_model_path)


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /aiffel/aiffel/project/FnGuide/kfdeberta-base and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 1-2 모델 이진분류로 수정

In [ ]:
model.to(device)

### 1-2-1. amphora/KorFinASC-XLM-RoBERTa용

In [65]:
# 출력 크기 변경
model.classifier.out_proj = torch.nn.Linear(in_features=1024, out_features=2, bias=True) 

In [ ]:
#재확인
model.to(device)

# 3. 데이터 전처리
## 3-1. 컬럼 삭제 및 레이블 변환

불용어

In [33]:
stopwords = [] 
stop_path = os.getenv('HOME') + '/aiffel/project/FnGuide/stopwords.txt'  # 자기에게 맞는 경로 설정

with open(stop_path, 'r', encoding='utf-8') as file:
    lines = file.read()

    sentences = re.findall(r'"([^"]*)"', lines)  # 작은따옴표로 둘러싸인 내용 추출
    for sentence in sentences:
        stopwords.append(sentence)

In [73]:
def transform(sentence, stopwords=None):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'\([^)]*\)', r'', sentence) #괄호로 둘러싸인 부분 제거
    sentence = re.sub(r'#\w+', '', sentence) #해쉬태그 문자 제거
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence) #문장 내의 구두점을 공백과 함께 분리(숫자 없앰 -> 금융이라 중요하다 생각)
    sentence = re.sub(r'[^a-zA-Z가-힣0-9?.!,% ]+', r' ', sentence) #영문 알파벳, 한글, 숫자, 구두점을 제외한 모든 문자를 제거
    sentence = re.sub(r"['\n']+", r"", sentence) #개행 문자 제거
    sentence = re.sub(r'[" "]+', " ", sentence) #연속된 공백을 하나의 공백으로 변환
    sentence = sentence.strip()

    if stopwords:
        words = sentence.split()
        filtered_words = [word for word in words if word not in stopwords]
        sentence = ' '.join(filtered_words)

    return sentence

In [68]:
def preprocess_data(df, data_type, stopwords=None):
    df = df[['document', 'label']]
    
    if data_type=='train':
        df = df.drop_duplicates().reset_index(drop=True)

#################표준어 체크

    new_sentences = []

    for sentence in tqdm(df['document']):
        sentence = transform(sentence, stopwords)
        try:
            sentence = sentence.replace('\\', '')
            spelled_sent = spell_checker.check(sentence)
            new_sentence = spelled_sent.checked
            
            if not new_sentence:
                new_sentence = sentence
                
            new_sentences.append(new_sentence)
                
        except:
            
            new_sentences.append(sentence)

    df['document'] = new_sentences
    
    if data_type == 'train':
        df = df.drop_duplicates().reset_index(drop=True)

    return df

In [64]:
train_df = load_data('modified_train.csv')
val_df = load_data('ratings_val.csv')
test_df = load_data('ratings_test.csv')

In [66]:
train_df = preprocess_data(train_df, data_type='train', stopwords=stopwords)
val_df = preprocess_data(val_df, data_type='val', stopwords=stopwords)
test_df = preprocess_data(test_df, data_type='test', stopwords=stopwords)

100%|██████████| 679/679 [00:59<00:00, 11.37it/s]
/tmp/ipykernel_31/4293808085.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['document'] = new_sentences
100%|██████████| 679/679 [00:57<00:00, 11.72it/s]


### 3-1-1. Back translate

In [37]:
def back_translate(data: pd.DataFrame, label: int, num: int, mode: str="back_translate", target_language: str='en', prob: int=0.4)-> pd.DataFrame:
    """
        Input:
            data (pandas.DataFrame) : CSV 파일로 구성된 데이터
            label (int) : 증강에 사용할 레이블
            num (int) : 증강 개수
            mode (str, optional) : 데이터 증강 모드. (back_translate/noise_add), 기본값은 "back_translate"
            target_language (str, optional) : 번역할 언어. 기본값은 'en' (영어)
            prob (float, optional) : noise_add 모드에서의 적용 확률
        
        Returns:
            augmented_df (pd.DataFrame) : 증강된 데이터들로 구성된 DataFrame  
    """
    
    # TextAugmentation 객체 생성
    agent = TextAugmentation(tokenizer="mecab", num_processes=1)

    # 선택한 레이블로 데이터 필터링
    filtered_data = data[data['label'] == label]['document'].tolist()

    # 데이터 증강
    augmented_data = []
    selected_data_set = set()  # 중복 선택된 데이터를 추적하기 위한 집합

    for _ in range(num):
        selected_data = random.choice(filtered_data)  # 원본 데이터 중에서 무작위로 선택

        while selected_data in selected_data_set:
            selected_data = random.choice(filtered_data)  # 중복된 데이터가 선택되면 다시 선택

        selected_data_set.add(selected_data)  # 선택된 데이터를 집합에 추가

        # 데이터 증강
        if mode == "noise_add":
            generated_data = agent.generate(selected_data, mode="noise_add", prob=prob, noise_mode=['phonological_change', 'vowel_change', 'jamo_split'])
        else:
            generated_data = agent.generate(selected_data, mode="back_translate", target_language=target_language)  #일본어로 하려면 target_language='jp'
        
        augmented_data.append(generated_data)

    # 생성된 데이터를 DataFrame으로 변환
    augmented_df = pd.DataFrame({'document': augmented_data, 'label': label})
    
    return augmented_df

### 3-1-2. Document Mixing

In [38]:
min_blocks = 2

In [39]:
def tokenize_sentences(text: str, mode: str ='kss') -> list :
    """
    문장을 분리하는 함수
    
    Input:
        text (str): 증강할 문장
        mode (str): 문장을 분리하는 방식

    Output:
        sentences(list): 분리된 문장 리스트
    """
    if mode == 'to':
        sentences = tokenizer.tokenize(text)  # 토크나이저를 사용하여 문장 단위로 분리

    else:
        sentences = kss.split_sentences(text)  # Split sentences using kss library

    return sentences

In [41]:
def mix_document(data, label, num, min_blocks = min_blocks, mix_ratio_1: int=0.3, mix_ratio_2: int=0.3, mode='kss')->pd.DataFrame:
    """
        Input:
            data (pandas.DataFrame) : CSV 파일로 구성된 데이터
            label (int) : 증강에 사용할 레이블
            num (int) : 증강 개수
            min_blocks (str): 최소 문장 수
            mix_ratio_1 (int) : 첫번째 문장에 제거할 비율
            mix_ratio_2 (int) : 두번쨰 문장에서 삽입할 비율
            mode (str, optional) : 데이터 증강 모드. (kss/tokenize), 기본값은 "back_translate"
            target_language (str, optional) : 번역할 언어. 기본값은 'en' (영어)
            prob (float, optional) : noise_add 모드에서의 적용 확률
        
        Returns:
            augmented_df (pd.DataFrame) : 증강된 데이터들로 구성된 DataFrame  
    """
    augmented_data = []
    filtered_data = data[data['label'] == label]['document'].tolist()
    
    selected_data_set = set()  # 중복 선택된 쌍을 추적하기 위한 집합

    for _ in range(num):
        selected_data = random.sample(filtered_data, 2)  # 원본 데이터 중에서 2개를 무작위로 선택

        while tuple(selected_data) in selected_data_set:
            selected_data = random.sample(filtered_data, 2)  # 중복된 데이터가 선택되면 다시 선택

        selected_data_set.add(tuple(selected_data))

        sentences_1 = tokenize_sentences(selected_data[0], mode)  # 첫 번째 선택된 데이터의 문장으로 분리
        sentences_2 = tokenize_sentences(selected_data[1], mode)  # 두 번째 선택된 데이터의 문장으로 분리

        # sentences_1에서 특정 비율만큼 문장 제거
        num_sentences_to_remove = int(len(sentences_1) * mix_ratio_1)
        removed_sentences = random.sample(sentences_1, num_sentences_to_remove)
        augmented_sentences_1 = [sentence for sentence in sentences_1 if sentence not in removed_sentences]

        
        # sentences_2에서 특정 비율만큼 문장 추출
        num_sentences_to_extract_2 = int(len(sentences_2) * mix_ratio_2)
        extracted_sentences_2 = random.sample(sentences_2, num_sentences_to_extract_2)
        
        # 추출된 문장들을 랜덤하게 위치에 삽입하여 augmented_sentences_1에 삽입
        for sentence in extracted_sentences_2:
            random_index = random.randint(0, len(augmented_sentences_1))
            augmented_sentences_1.insert(random_index, sentence)
        
        if mode == 'to':
            augmented_text = tokenizer.convert_tokens_to_string(augmented_sentences_1)  # 토큰을 다시 문자열로 변환하여 문서로 만듦
        else:
            augmented_text = ' '.join(augmented_sentences_1)  # 문장 내의 단어들을 공백으로 연결하여 문장으로 만듦

        augmented_data.append(augmented_text)
        
    # 생성된 증강 데이터에 레이블 1 추가
    augmented_df = pd.DataFrame({'document': augmented_data, 'label': 1})
    
    # 원래 데이터에 추가
#     df = pd.concat([data, augmented_df], ignore_index=True)
    
    return augmented_df

### 3-1-3. KoEDA

p = (alpha_sr, alpha_ri, alpha_rs, prob_rd)

In [43]:
from koeda import EDA

def ag_koeda(data: pd.DataFrame, label: int, num: int, p: tuple=(0.3, 0.3, 0.3, 0.3))-> pd.DataFrame:
    """
    Input:
        data (DataFrame): 증강시킬 데이터가 있는 데이터 프레임
        label (int): 증강시키고 싶은 label
        num (int): 증강시킬 데이터의 수
        p (tuple): alpha_sr, alpha_ri, alpha_rs, prob_rd 기법의 수치
        
    Output:
        augmented_df (DataFrame): 증강시킨 데이터가 있는 데이터 프레임
    """
    
    eda = EDA(morpheme_analyzer="Mecab", alpha_sr=p[0], alpha_ri=p[1], alpha_rs=p[2], prob_rd=p[3])

    # 데이터 필터링
    filtered_data = data[data['label'] == label]['document'].tolist()
    
    selected_data_set = set()  # 중복 선택된 데이터를 추적하기 위한 집합

    # 데이터 증강
    augmented_data = []
    for _ in range(num):
        selected_data = random.choice(filtered_data)  # 원본 데이터 중에서 무작위로 선택

        while selected_data in selected_data_set:
            selected_data = random.choice(filtered_data)  # 중복된 데이터가 선택되면 다시 선택

        selected_data_set.add(selected_data)  # 선택된 데이터를 집합에 추가

        # 데이터 증강
        generated_data = eda(selected_data, p=p, repetition=1)  # repetition: 한 문장을 몇 개로 증강시킬 것인지
        augmented_data.append(generated_data)

    # 생성된 데이터를 DataFrame으로 변환
    augmented_df = pd.DataFrame({'document': augmented_data, 'label': label})

    return augmented_df

### 3-1-4. Fill Masked Model

In [44]:
fill_model = AutoModelForMaskedLM.from_pretrained(model_name)
fill_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [69]:
def fill_masked_augmentation(df, label_num, random_number, num_masks_ratio, max_length=512):
    """
    Input:
        df (DataFrame): 증강시킬 데이터가 있는 데이터 프레임
        label_num (int): 증강시키고 싶은 label number
        random_number (int): 증강시킬 데이터의 수
        num_masks_ratio (float): 본문에서 마스킹 시킬 비율
        max_length (int): sentence 토큰의 최대 길이
        
    Output:
        aug_data (DataFrame): 증강시킨 데이터가 있는 데이터 프레임
    """
    label_data = df[df['label'] == label_num]
    random_sentences = random.sample(label_data['document'].tolist(), random_number)
    predicted_tokens_list = []
    fill_model.to(device)

    for sentence in tqdm(random_sentences):
        num_masks = int(len(sentence.split()) * num_masks_ratio)
        words = sentence.split()
        masked_indices = random.sample(range(len(words)), num_masks)
        masked_text = ' '.join('[MASK]' if i in masked_indices else word for i, word in enumerate(words))
        tokenized_sentence = fill_tokenizer.tokenize(masked_text)
        masked_indices = [i for i, token in enumerate(tokenized_sentence) if token == fill_tokenizer.mask_token]

        # 문장 길이 체크 및 잘라내기
        if len(tokenized_sentence) > max_length:
            continue

        predicted_tokens = []
        for index in masked_indices:
            tokens = tokenized_sentence.copy()
            tokens[index] = fill_tokenizer.mask_token
            input_ids = fill_tokenizer.convert_tokens_to_ids(tokens)
            inputs = torch.tensor([input_ids]).to(device)

            with torch.no_grad():
                outputs = fill_model(inputs)
                predictions = outputs.logits.argmax(dim=-1)

            predicted_token = fill_tokenizer.convert_ids_to_tokens(predictions[0, index].item())
            predicted_tokens.append(predicted_token)

        new_sentence = tokenized_sentence.copy()

        for index, predicted_token in zip(masked_indices, predicted_tokens):
            new_sentence[index] = predicted_token

        new_sentence = fill_tokenizer.convert_tokens_to_string(new_sentence)
        predicted_tokens_list.append(new_sentence)

    aug_data = pd.DataFrame({'document': predicted_tokens_list, 'label': label_num})

    return aug_data

### 3-1-5. Summarize sentences

In [48]:
def summarize_sentences(data: pd.DataFrame, label: int, num: int, sum_ratio: int=0.8)-> pd.DataFrame:
    """
    Input:
        data (DataFrame): 증강시킬 데이터가 있는 데이터 프레임
        label (int): 증강시키고 싶은 label
        num (int): 증강시킬 데이터의 수
        sum_ratio (int): 요약할 문장 비율
        
    Output:
        augmented_df (pd.DataFrame): 증강시킨 데이터가 있는 데이터 프레임
    """
    
    # 데이터 필터링
    filtered_data = data[data['label'] == label]['document'].tolist()
    
    selected_data_set = set()  # 중복 선택된 데이터를 추적하기 위한 집합

    # 데이터 증강
    augmented_data = []
    
    for _ in range(num):
        selected_data = random.choice(filtered_data)  # 원본 데이터 중에서 무작위로 선택

        while selected_data in selected_data_set:
            selected_data = random.choice(filtered_data)  # 중복된 데이터가 선택되면 다시 선택

        selected_data_set.add(selected_data)  # 선택된 데이터를 집합에 추가
        
        sentences = tokenize_sentences(selected_data)
        
        sen_num = int(len(sentences) * sum_ratio)
        
        # 데이터 증강
        generated_data = kss.summarize_sentences(selected_data, max_sentences=sen_num)
        
        # 요약된 문장들을 하나의 이어진 문장으로 변환  -> kss.summarize_sentences결과가 ['ab','df','fg'] 이런식으로 나오기 때문
        if generated_data:
            augmented_text = ' '.join(generated_data)
        else:
            augmented_text = ''
        
        # 생성된 데이터를 리스트에 추가
        augmented_data.append(augmented_text)

    # 생성된 데이터를 DataFrame으로 변환
    augmented_df = pd.DataFrame({'document': augmented_data, 'label': label})

    return augmented_df

## 3-2. 클래스 및 함수 만들기

In [16]:
class TrainTestDataset(Dataset):
    def __init__(self, dataframe, transform=True, train=True):
        self.data = dataframe
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.train:  # train data이면 sentence와 label 같이 전처리
            labels = self.data['label'][index]
            sentence = self.data['document'][index]
        else:  # test data이면 label만 같이 전처리
            sentence = self.data['document'][index]

        encoded_dict = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=config.MAX_LEN,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
#             return_token_type_ids=True
        )

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]

        if self.train:
            target = torch.tensor(labels)
            sample = (padded_token_list, token_type_id, att_mask, target)
        else:
            sample = (padded_token_list, token_type_id, att_mask)

        return sample

# 4. 데이터 만들기

In [ ]:
#wandb 기록용
hyperparameters={
    "model": "klue/roberta-base",
    "data_ag_method" : "False",
    "Stopword" : "True",    
    "MAX_LEN" : 512,
    "epochs" : 3,
    "BATCH_SIZE" : 4,
    'NUM_CORES': 0,
    'learning_rate': 1e-5,
    'eps' : 1e-8,
    'loss' : 'BalancedCrossEntropyLoss'
      }

#wandb 초기화
wandb.init(project='fn_guide_project',name='AD_NEWS_klue/roberta-base', config = hyperparameters)
config = wandb.config

In [16]:
train_dataset = TrainTestDataset(train_df, train=True)
val_dataset = TrainTestDataset(val_df, train=True)
test_dataset = TrainTestDataset(test_df, train=False)

In [17]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size=config.BATCH_SIZE,
                                                shuffle=True,
                                                num_workers=config.NUM_CORES)

val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                                batch_size=config.BATCH_SIZE,
                                                shuffle=False,
                                                num_workers=config.NUM_CORES)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                                batch_size=config.BATCH_SIZE,
                                                shuffle=False,
                                                num_workers=config.NUM_CORES)

In [18]:
model.to(device)
optimizer = AdamW(model.parameters(), lr=config.learning_rate, eps =config.eps)

# 5. 학습

In [17]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (self.alpha * (1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [18]:
class Balanced_crossentropy_loss(nn.Module):
    def __init__(self, beta=0.5):
        super(BalancedCrossEntropyLoss, self).__init__()
        self.beta = beta

    def forward(self, inputs, targets):
        # inputs: 모델의 출력 logits
        # targets: 실제 레이블 (0 또는 1)

        # 확률값으로 변환하여 클래스 1에 해당하는 값만 선택
        prob = torch.softmax(inputs, dim=1)
        prob_class_1 = prob[:, 1]

        # 클래스별 샘플 수 계산
        class_count = torch.bincount(targets, minlength=2)  # 이진 분류에서는 minlength=2로 설정합니다.

        # 클래스 빈도에 반비례하는 가중치 계산
        weights = 1.0 / (class_count.float() + 1e-8)

        # 클래스별 가중치를 적용하여 손실 계산
        weight = weights[targets]

        # Balanced Cross Entropy Loss 계산
        loss = -weight * prob_class_1

        # 모든 샘플에 대한 손실 평균 계산
        loss = torch.mean(loss)

        return loss

In [19]:
class Weighted_crossentropy_loss(nn.Module):
    def __init__(self, pos_weight=None, weight=None, reduction='mean'):
        super(BinaryWeightedCrossEntropyLoss, self).__init__()
        self.pos_weight = pos_weight
        self.weight = weight
        self.reduction = reduction

    def forward(self, inputs, targets):
        targets_one_hot = torch.eye(2)[targets].to(inputs.device)  # one-hot 인코딩, inputs와 동일한 디바이스로 이동
        weight_tensor = torch.tensor(self.weight).to(inputs.device)  # weight를 Tensor로 변환, inputs와 동일한 디바이스로 이동
        pos_weight_tensor = torch.tensor(self.pos_weight).to(inputs.device)  # pos_weight를 Tensor로 변환, inputs와 동일한 디바이스로 이동
        loss = F.binary_cross_entropy_with_logits(inputs, targets_one_hot, pos_weight=pos_weight_tensor, weight=weight_tensor, reduction=self.reduction)
        return loss


In [ ]:
import wandb
import random

output = os.getenv('HOME')+'/aiffel/project/FnGuide/model/' #자신의 경로로 바꾸기
model_name1 = model_name.replace('/','_')

# loss_fn = Weighted_crossentropy_loss(pos_weight=3, weight=1)
# loss_fn = Balanced_crossentropy_loss(beta=0.5)

model.train()

# wandb.watch를 통해 모델 가중치를 기록
wandb.watch(model)

# 학습 및 검증 결과를 로깅할 리스트 초기화
losses = []
accuracies = []

# 학습 루프
for i in range(config.epochs):
    total_loss = 0.0
    correct = 0
    total = 0

    # Training loop
    for input_ids_batch, token_type_id_batch, attention_masks_batch, y_batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        input_ids_batch = input_ids_batch.to(device)
        token_type_id_batch = token_type_id_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        y_pred = model(input_ids_batch, token_type_ids=token_type_id_batch, attention_mask=attention_masks_batch)[0].to(device)

        loss = F.cross_entropy(y_pred, y_batch)  # 일반 loss 사용
#         loss = loss_fn(y_pred, y_batch)   # 다른 loss 사용

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

    losses.append(total_loss)
    accuracies.append(correct.float() / total)

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_correct = 0
        val_total = 0

        for val_input_ids_batch, val_token_type_id_batch, val_attention_masks_batch, val_y_batch in tqdm(val_dataloader):
            val_y_batch = val_y_batch.to(device)
            val_input_ids_batch = val_input_ids_batch.to(device)
            val_token_type_id_batch = val_token_type_id_batch.to(device)
            val_attention_masks_batch = val_attention_masks_batch.to(device)
            val_y_pred = model(val_input_ids_batch, token_type_ids=val_token_type_id_batch, attention_mask=val_attention_masks_batch)[0].to(device)

            _, val_predicted = torch.max(val_y_pred, 1)
            val_correct += (val_predicted == val_y_batch).sum()
            val_total += len(val_y_batch)

        val_accuracy = val_correct.float() / val_total
        print("Train Loss:", total_loss / total, "Accuracy:", correct.float() / total)
        print("Validation Accuracy:", val_accuracy)

    

    # wandb에 메트릭 로깅
    wandb.log({"Train Loss": total_loss / total, "Train Accuracy": correct.float() / total, "Validation Accuracy": val_accuracy})
    model.train()

    # 모델 저장
    torch.save(model.state_dict(), os.path.join(output, f'{model_name1}_{config.data_ag_method}_{config.loss}_stopword_classification_model_{i+1}.pt')) #모델 저장이름도 자기에게 맞게 바꾸기
    wandb.save(os.path.join(output, f'{model_name1}_{config.data_ag_method}_{config.loss}_stopword_classification_model_{i+1}.pt'))  # 모델 wandb에 저장

# 6. 평가

In [ ]:
model.eval()

pred = []

with torch.no_grad():
    for input_ids_batch, token_type_id_batch, attention_masks_batch in tqdm(test_dataloader):
        y_pred = model(input_ids_batch.to(device),token_type_ids = token_type_id_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        _, predicted = torch.max(y_pred, 1)
        pred.extend(predicted.tolist())


    accuracy = accuracy_score(pred, test_df['label'])
    print(accuracy_score(pred, test_df['label']))
    classification_report_str = classification_report(test_df['label'], pred, target_names=['real_news', 'AD'])
    print(classification_report(test_df['label'], pred, target_names=['real_news', 'AD']))

    cm = confusion_matrix(test_df['label'], pred)
    sns.heatmap(cm, annot = True, cmap='coolwarm', xticklabels=['real_news', 'AD'], yticklabels=['real_news', 'AD'], fmt='d')
    plt.show()
    
    # f1-score 추출
    report = classification_report(test_df['label'], pred, target_names=['real_news', 'AD'], output_dict=True)

    wandb.log({"test_accuracy": accuracy})
    wandb.log({"classification_report": classification_report_str})
    wandb.log({"confusion_matrix": wandb.plot.confusion_matrix(probs=None,
                                                                y_true=test_df['label'],
                                                                preds=pred,
                                                                class_names=['real_news', 'AD'])})
    
wandb.finish()

## 6-1. 틀린 문장 체크

In [ ]:
test_check_df = test_df.copy() #테스트 데이터프레임 복사
pred1 = pd.Series(pred, name="pred") #예측된 레이블 시리즈로 만들고 이름 붙혀주기
test_check_df = pd.concat([test_check_df, pred1], axis = 1) #두개를 하나의 데이터프레임으로 합치기

diff_indices = test_check_df[test_check_df['label'] != test_check_df['pred']].index #답변이 틀린것들의 index를 뽑을 수 있음
diff_indices
#test_check_df.to_csv(data_path +'test_check_df.csv') #혹시나 저장하려면 data_path 잘 확인해서 저장하기

In [122]:
def wrong_all_check(diff_indices):
    for i in diff_indices:
        print("인덱스:", i)
        print('실제 답변:{}, 예측 답변:{}'.format(test_check_df['label'][i],test_check_df['pred'][i]))
        print('document: \n',test_check_df['document'][i])
        print('\n')

In [ ]:
wrong_all_check(diff_indices)

----------------------